In [6]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import time
import json
import yfinance as yf

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Defining tools
function_list = {}

def get_share_values(share_list: list):
    print(f'Getting value of {share_list}')
    result_dict = {}
    for share in list(share_list):
        share_name = share
        share_value = yf.Ticker(share).info['currentPrice']
        result_dict[share_name] = share_value

    return json.dumps(result_dict)

function_list['get_share_values'] = get_share_values

def is_function_call(message: str):
    if message.choices[0].finish_reason == 'tool_calls':
        return True
    return False

def call_gpt(message):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools=tools,
    temperature=0,
    max_tokens=1000,
    messages=message,
    )
    return response



tools = [
    {
        "type": "function",
        "function": {
            "name": "get_share_values",
            "description": "Get the value of shares of a company",
            "parameters": {
                "type": "object",
                "properties": {
                    "share_list": {
                        "type": "array",
                        "items": {
                                "type": "string"
                        },
                        "description":"List of all company stock ticker symbols in the stock market, e.g., Google, [GOOG, GOOGL]",
                    }
                },
                "required": ["share_list"],
            },
        }
    }
]


client = OpenAI()




In [136]:

print("Chat : Bem vindo ao chatbot do neutão") 
log_conversation = []
input_msg = str(input("user: "))
# if input_msg == 'exit':
#    break
print(f'Neutão: {input_msg}')
log_conversation.append({"role": "user", "content": input_msg})
stream = call_gpt(log_conversation)
log_conversation.append(stream.choices[0].message)
print(f'{stream = }')
if is_function_call(stream):
    for tool in stream.choices[0].message.tool_calls:
        function_name = function_list[tool.function.name]
        function_args = json.loads(tool.function.arguments)
        function_return = function_name(**function_args)
        log_conversation.append({
            "role":"tool", 
            "tool_call_id":tool.id, 
            "name": tool.function.name, 
            "content":function_return
        })
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=log_conversation,)


print(f' {second_response.choices[0].message.content}')
# log_sys_response = ""
# for message in stream:
#     if message.choices[0].delta.content is not None:
#         print(message.choices[0].delta.content,end='')
#     log_sys_response = log_sys_response + str(message.choices[0].delta.content)
# log_conversation.append({"role": "assistant", "content": log_sys_response})
print('\n')
    






Chat : Bem vindo ao chatbot do neutão
Neutão: qual o valor das ações da google
stream = ChatCompletion(id='chatcmpl-9lh6Q6VGKDPI3mGeixdLreEnoykjQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_IrEPwP6S8QIi60cxA8hxogui', function=Function(arguments='{"share_list":["GOOG","GOOGL"]}', name='get_share_values'), type='function')]))], created=1721154130, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=84, total_tokens=105))
Getting value of ['GOOG', 'GOOGL']
 O valor das ações da Google é de $185.435 para a GOOG e $183.72 para a GOOGL.




In [100]:
stream.choices[0].finish_reason == ''


True

2.165